Porcentaje de personas que se dedican a actividades de cuidados y no reciben remuneración

In [2]:
##
rm(list=ls())
#Paquetes y liberías
if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, srvyr, readr, stringr, dplyr, utils, survey)
# URL del archivo ZIP
url <- "https://www.inegi.org.mx/contenidos/programas/enigh/nc/2022/microdatos/enigh2022_ns_poblacion_csv.zip"

Cargando paquete requerido: pacman

Warning message:
"package 'pacman' was built under R version 4.4.2"


In [3]:
##Creación de directorio temporal
td<- tempdir()
# Descarga del archivo temporal
tf = tempfile(tmpdir=td, fileext=".zip")
download.file(url, tf)
unzip(tf, files="poblacion.csv", exdir=td, 
      overwrite=TRUE)
fpath=file.path(td,"poblacion.csv")
unlink(td)

In [ ]:
pob <- read_csv(fpath, show_col_types = FALSE)


In [5]:
datos <- pob[c("folioviv", "foliohog", "hor_4", "min_4", "est_dis", "entidad", "edad", "upm", "factor")]

datos <- datos %>%
  filter(entidad == "09") %>%
  mutate(
    pob_interes = case_when(
      (hor_4 != 0 | !is.na(hor_4)) ~ 1,
      (hor_4 == 0 & min_4 != 0 & !is.na(min_4)) ~ 1,
      TRUE ~ 0
    ) 
  ) 
head(datos,10)

folioviv,foliohog,hor_4,min_4,est_dis,entidad,edad,upm,factor,pob_interes
<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>
0900006901,1,28,0,141,09,66,0003071,1578,1
0900006901,1,NA,NA,141,09,30,0003071,1578,0
0900006901,1,NA,NA,141,09,32,0003071,1578,0
0900006901,1,NA,NA,141,09,8,0003071,1578,0
0900006901,1,NA,NA,141,09,5,0003071,1578,0
0900006902,1,NA,NA,141,09,71,0003071,1578,0
0900006902,1,NA,NA,141,09,68,0003071,1578,0
0900006902,1,NA,NA,141,09,46,0003071,1578,0
0900006902,1,NA,NA,141,09,25,0003071,1578,0


In [6]:
## Diseño muestral
dm <- datos %>%
  as_survey_design(
    ids = upm,
    strata = est_dis,
    weights = factor,
    nest = TRUE)

In [7]:
pob_cuidados<- dm|>
  group_by(pob_interes)|>
  summarise(pob_cuid = survey_total(vartype = "cv"),
            porc_cuid = survey_prop(vartype = "cv")*100)|>
  #mutate(prc_p_cuid =pob_cuid*100/sum(pob_cuid))|> PUEDES OBTENER EL PORCENTAJE DE FORMA TRADICIONAL
  filter(pob_interes == 1)

In [8]:
head(pob_cuidados)


pob_interes,pob_cuid,pob_cuid_cv,porc_cuid,porc_cuid_cv
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1569286,0.05277464,16.78938,4.772984
